In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time

from sklearn.preprocessing import OneHotEncoder

STYLE = "#ffffff"

%load_ext autoreload
%autoreload 2

In [2]:
class Sculptor():
    
    def __init__(self, void_dim,
                 n_edge_elements,
                 n_plane_elements,
                 n_volume_elements,
                 color_edges,
                 color_planes,
                 color_volumes,
                 element_edge_min,
                 element_edge_max,
                 element_grid_min,
                 element_grid_max,
                 element_plane_min,
                 element_plane_max,
                 element_volume_min,
                 element_volume_max,
                 step,
                 verbose):
        
        self.void = np.zeros((void_dim, void_dim, void_dim))
        self.color_void = np.empty(self.void.shape, dtype=object)
        self.colors = np.empty(self.void.shape, dtype=object)
        
        self.color_edges = color_edges
        self.color_planes = color_planes
        self.color_volumes = color_volumes
        
        self.n_edge_elements = n_edge_elements
        self.n_plane_elements = n_plane_elements
        self.n_volume_elements = n_volume_elements
        self.style = "#ffffff"
        
        self.element_edge_min= element_edge_min
        self.element_edge_max = element_edge_max
        self.element_grid_min= element_grid_min
        self.element_grid_max = element_grid_max
        self.element_plane_min = element_plane_min
        self.element_plane_max = element_plane_max
        self.element_volume_min = element_volume_min
        self.element_volume_max = element_volume_max
        self.step = step
        
        self.verbose = verbose
        
        
    def return_axis(self): # CAN I JUST HAVE ONE????
        
        self.section = np.random.randint(low=0-1, high=self.void[0].shape[0]) # selects a section of the axis to work on
        
        self.axis_selection = np.random.randint(low=0, high=3) # decides if we are working in the X, Y or Z axis
        
        if self.axis_selection == 0:
            self.working_plane = self.void[self.section,:,:]
            self.color_parameters = self.color_void[self.section,:,:]
        elif self.axis_selection == 1:
            self.working_plane = self.void[:,self.section,:]
            self.color_parameters = self.color_void[:,self.section,:] 
        elif self.axis_selection == 2:
            self.working_plane = self.void[:,:,self.section]
            self.color_parameters = self.color_void[:,:,self.section]
        else:
            print("error")
            
        return self.working_plane, self.color_parameters # returns the selected plane for the void and the parameters array
    
    ### MAIN FUNCTIONS ###

    ######################
    ## BUILDS THE EDGES ##
    ######################
    
    def add_edge(self): # WHAT TO DO WITH THE WORKING PLANE PARAMETER
        
        self.working_plane = self.return_axis()[0]
        self.color_parameters = self.return_axis()[1]
        # selection of the axis to work on
        
        if self.verbose == True:
            print(working_plane)
            print("###############################################################")

        #Variables
        self.edge_length = random.randrange(self.element_edge_min, self.element_edge_max, self.step) # estas variables quizas no necesiten ser self!!
        self.edge_plane = np.random.randint(low=0, high=2)

        if self.edge_plane == 0:
            self.element = np.ones(self.edge_length).reshape(self.edge_length,1)
        else:
            self.element = np.ones(self.edge_length).reshape(self.edge_length,1).T

        # creates the element to be inserted
        self.delta = np.array(self.working_plane.shape) - np.array(self.element.shape) 
        # finds the delta between the size of the void and the size of the element
        self.top_left_corner = np.array((np.random.randint(low=0, high=self.delta[0]) , np.random.randint(low=0, high=self.delta[1])))
        # finds the coordinates of the top left corner
        self.bottom_right_corner = np.array(self.top_left_corner) + np.array(self.element.shape) #- np.array([1,1]))
        # finds the coordinates of the bottom right corner
        
        self.working_plane[self.top_left_corner[0]:self.bottom_right_corner[0] , self.top_left_corner[1]:self.bottom_right_corner[1]] = self.element
        # makes the slides using the coordinates equal to the element
        
        self.color_parameters[self.top_left_corner[0]:self.bottom_right_corner[0] , self.top_left_corner[1]:self.bottom_right_corner[1]] = self.color_edges

        if self.verbose == True:
            print(self.working_plane)
            print("###############################################################")
        
        return self.void, self.color_void

    #######################
    ## BUILDS THE PLANES ##
    #######################
    
    def add_plane(self): # WHAT TO DO WITH THE WORKING PLANE PARAMETER
        
        self.element = None
        self.section = None
        self.delta = None
        self.top_left_corner = None
        self.bottom_right_corner = None
        self.working_plane = self.return_axis()[0]
        
        if self.verbose == True:
            print(self.working_plane)
            print("###############################################################")

        #Variables
        self.element = np.ones((random.randrange(self.element_plane_min, self.element_plane_max, self.step), random.randrange(self.element_plane_min, self.element_plane_max, self.step)))
        # creates the element to be inserted
        self.delta = np.array(self.working_plane.shape) - np.array(self.element.shape) 
        # finds the delta between the size of the void and the size of the element
        self.top_left_corner = np.array((np.random.randint(low=0, high=self.delta[0]) , np.random.randint(low=0, high=self.delta[1])))
        # finds the coordinates of the top left corner
        self.bottom_right_corner = np.array(self.top_left_corner) + np.array(self.element.shape) #- np.array([1,1]))
        # finds the coordinates of the bottom right corner
        self.working_plane[self.top_left_corner[0]:self.bottom_right_corner[0] , self.top_left_corner[1]:self.bottom_right_corner[1]] = self.element
        # makes the slides using the coordinates equal to the element
        
        self.color_parameters[self.top_left_corner[0]:self.bottom_right_corner[0] , self.top_left_corner[1]:self.bottom_right_corner[1]] = self.color_planes
        
        if self.verbose == True:
            self.print_information()
            print("###############################################################")
            
        return self.void, self.color_void

    ###########################
    ## BUILDS THE CANTILEVER ##
    ###########################
    
    def add_pipe_cantilever(self): # THIS IS GOOD!!
        
        self.element = None
        self.working_plane = None
        self.delta = None
        self.top_left_corner = None
        self.bottom_right_corner = None
        self.axis_selection = np.random.randint(low=0, high=2)
        self.shape_selection = np.random.randint(low=0, high=2)
        self.depth = random.randrange(self.element_volume_min, self.element_volume_max, self.step)
        
        if self.verbose == True:
            print(self.working_plane)
            print("###############################################################")
            
        self.element = np.ones((random.randrange(self.element_volume_min, self.element_volume_max, self.step), random.randrange(self.element_volume_min, self.element_volume_max, self.step)))
        self.element = np.repeat(self.element, repeats=self.depth, axis=0).reshape((self.element.shape[0],self.element.shape[1],self.depth))

        self.element_void = np.zeros((self.element.shape[0]-2, self.element.shape[1]-2))
        self.element_void = np.repeat(self.element_void, repeats=self.depth).reshape((self.element_void.shape[0],self.element_void.shape[1],self.depth))

        # element[1:-1,1:-1,:] = element_void # elegir pasar el vacio o no como parte del volumen
        
        self.delta = np.array(self.void.shape) - np.array(self.element.shape) # ENCONTRAR LOS NUEVOS DELTAS

        corner_1 = np.array((np.random.randint(low=0, high=self.delta[0]) , np.random.randint(low=0, high=self.delta[1]), np.random.randint(low=0, high=self.delta[2])))
        corner_2 = np.array((corner_1[0] + self.element.shape[0], corner_1[1], corner_1[2]))
        corner_3 = np.array((corner_1[0], corner_1[1], corner_1[2] + self.element.shape[2]))
        corner_4 = np.array((corner_1[0] + self.element.shape[0], corner_1[1], corner_1[2] + self.element.shape[2]))
        
        corner_5 = np.array((corner_1[0], corner_1[1] + self.element.shape[1], corner_1[2]))
        corner_6 = np.array((corner_2[0], corner_2[1] + self.element.shape[1], corner_2[2]))
        corner_7 = np.array((corner_3[0], corner_3[1] + self.element.shape[1], corner_3[2]))
        corner_8 = np.array((corner_4[0], corner_4[1] + self.element.shape[1], corner_4[2]))
        
        self.color_volume = np.random.randint(0,3)
        
        # creates the floor and ceiling
        self.void[corner_3[0]:corner_8[0], corner_3[1]:corner_8[1], corner_3[2]-1] = self.element[:,:,0]
        self.color_void[corner_3[0]:corner_8[0], corner_3[1]:corner_8[1], corner_3[2]-1] = self.color_volumes[self.color_volume]
        
        self.void[corner_1[0]:corner_6[0], corner_1[1]:corner_6[1], corner_1[2]] = self.element[:,:,1]
        self.color_void[corner_1[0]:corner_6[0], corner_1[1]:corner_6[1], corner_1[2]] = self.color_volumes[self.color_volume]
        
        # creates de walls
        if self.shape_selection ==0:
            if self.axis_selection == 0:
                self.void[corner_1[0], corner_1[1]:corner_7[1], corner_1[2]:corner_7[2]] = self.element[0,:,:]
                self.color_void[corner_1[0], corner_1[1]:corner_7[1], corner_1[2]:corner_7[2]] = self.color_volumes[self.color_volume]
                
                self.void[corner_2[0]-1, corner_2[1]:corner_8[1], corner_2[2]:corner_8[2]] = self.element[1,:,:]
                self.color_void[corner_2[0]-1, corner_2[1]:corner_8[1], corner_2[2]:corner_8[2]] = self.color_volumes[self.color_volume]
            else:
                self.void[corner_5[0]:corner_8[0], corner_5[1], corner_5[2]:corner_8[2]] = self.element[:,0,:]
                self.color_void[corner_5[0]:corner_8[0], corner_5[1], corner_5[2]:corner_8[2]] = self.color_volumes[self.color_volume]
                
                self.void[corner_1[0]:corner_4[0], corner_1[1], corner_1[2]:corner_4[2]] = self.element[:,0,:]
                self.color_void[corner_1[0]:corner_4[0], corner_1[1], corner_1[2]:corner_4[2]] = self.color_volumes[self.color_volume]
                
        else:
            if self.axis_selection == 0:
                self.void[corner_1[0], corner_1[1]:corner_7[1], corner_1[2]:corner_7[2]] = self.element[0,:,:]
                self.color_void[corner_1[0], corner_1[1]:corner_7[1], corner_1[2]:corner_7[2]] = self.color_volumes[self.color_volume]
                
                self.void[corner_5[0]:corner_8[0], corner_5[1], corner_5[2]:corner_8[2]] = self.element[:,0,:]
                self.color_void[corner_5[0]:corner_8[0], corner_5[1], corner_5[2]:corner_8[2]] = self.color_volumes[self.color_volume]
            else:
                self.void[corner_2[0]-1, corner_2[1]:corner_8[1], corner_2[2]:corner_8[2]] = self.element[1,:,:]
                self.color_void[corner_2[0]-1, corner_2[1]:corner_8[1], corner_2[2]:corner_8[2]] = self.color_volumes[self.color_volume]
                
                self.void[corner_1[0]:corner_4[0], corner_1[1], corner_1[2]:corner_4[2]] = self.element[:,0,:]
                self.color_void[corner_1[0]:corner_4[0], corner_1[1], corner_1[2]:corner_4[2]] = self.color_volumes[self.color_volume]
        
        if self.verbose == True:
            self.print_information()
            print("###############################################################")
        
        
        
        return self.void
    
    #####################
    ## BUILDS THE GRID ##
    #####################
    
    def add_grid(self):
        self.working_plane = self.void[:,:,self.section]
        self.color_parameters = self.return_axis()[1]
        # selection of the axis to work on
        
        if self.verbose == True:
            print(working_plane)
            print("###############################################################")

        column_height = random.randrange(self.element_grid_min, self.element_grid_max, self.step)
        element = np.ones(column_height).reshape(column_height,1)
        
        x = np.arange(start=1, stop=49, step=9)
        y = np.arange(start=1, stop=49, step=9)
        
        z_base = np.zeros((6,))
        z_top = z_base + column_height
        
        grid_coor_base = np.array(np.meshgrid(x, y, z_base)).T[0]
        grid_coor_top = np.array(np.meshgrid(x, y, z_top)).T[0]
        grid_coor = np.concatenate((grid_coor_base, grid_coor_top), axis = 2).astype(int)
        
        for column_row in grid_coor:
            for column in column_row:
                self.void[column[0]:column[3]+1, column[1]:column[4]+1, column[2]:column[5]] = element.reshape((column_height,))
        
        for column_row in grid_coor:
            for column in column_row:
                self.color_void[column[0]:column[3]+1, column[1]:column[4]+1, column[2]:column[5]] = self.color_edges

        if self.verbose == True:
            print(self.working_plane)
            print("###############################################################")
        
        return self.void, self.color_void
    
    ### ULTILS ###
    
    def print_information(self):
        print(f"void shape is: {np.array(self.void[0].shape)}")
        print(f"element shape is : {np.array(self.element.shape)}")
        print(f"the axis selection is: {self.axis_selection}")
        print(f"delta is: {self.delta}")
        print(f"section is: {self.section}")
        print(f"top left corner is: {self.top_left_corner}")
        print(f"bottom right corner is: {self.bottom_right_corner}")
        print(f"slices are: {self.top_left_corner[0]}:{self.bottom_right_corner[0]} and {self.top_left_corner[1]}:{self.bottom_right_corner[1]}")
        print("###############################################################")
    
    def plot_sections(self):
        sculpture = self.void
        fig, axes = plt.subplots(ncols=6, nrows=int(np.ceil(self.void.shape[0]/6)), figsize=(25, 25), facecolor = (self.style))
        axes = axes.ravel() # flats
        for index in range(self.void.shape[0]):
            axes[index].imshow(sculpture[index,:,:], cmap = "gray")
      
    
    def plot_sculpture(self): # add call to generative sculpt and then plot like 12
        #sculpture = self.void
        fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(25, 25), facecolor = (self.style), subplot_kw=dict(projection="3d"))
        axes = axes.ravel() # flats
        for index in range(1):
            self.void = np.zeros((void_dim, void_dim, void_dim))
            sculpture = self.generative_sculpt()
            axes[index].voxels(sculpture[0], facecolors=sculpture[1], edgecolors="k", linewidth=0.05) # axes[index]
        plt.savefig('image.png') # agregar tiempo de impresion y exportar 3D
    
    ### GENERATOR ###
    
    def generative_sculpt(self):
        start = time.time()
        for edge in range(self.n_edge_elements):
            self.add_edge()
            
        for plane in range(self.n_plane_elements):
            self.add_plane()
            
        for volume in range(self.n_volume_elements):
            self.add_pipe_cantilever()
        
        for volume in range(self.n_volume_elements):
            self.add_grid()
            
        print ('Time for sculptures is {} sec'.format(time.time()-start))
        
        return self.void, self.color_void

In [3]:
void_dim = 48

# greenyellow, chartreuse, mediumpurple, orange, crimson, turquoise, gold

sculptor = Sculptor(void_dim = void_dim,
                    n_edge_elements = 0,
                    n_plane_elements = 8,
                    n_volume_elements = 6,
                    color_edges = "dimgrey",
                    color_planes = "snow",
                    color_volumes = ["crimson", "turquoise", "gold"], # ["greenyellow","orange","mediumpurple"]
                    element_edge_min= int(void_dim*0.8),
                    element_edge_max = int(void_dim*0.9),
                    element_grid_min = 18, # 40,
                    element_grid_max = 20, # 45,
                    element_plane_min = int(void_dim*0.4),
                    element_plane_max = int(void_dim*0.8),
                    element_volume_min = int(void_dim*0.2),
                    element_volume_max = int(void_dim*0.5),
                    step = 1,
                    verbose = False)

sculpture = sculptor.generative_sculpt()

# sculptor.plot_sculpture()

Time for sculptures is 0.009032011032104492 sec


In [4]:
sculpture[1].shape

(48, 48, 48)

In [5]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

colors = ohe.fit_transform(sculpture[1].reshape(-1, 1))

encoded = colors.reshape((void_dim,void_dim,void_dim,ohe.categories_[0].size))

decoded = ohe.inverse_transform(encoded.reshape((void_dim*void_dim*void_dim,ohe.categories_[0].size)))

decoded_colors = decoded.reshape((void_dim,void_dim,void_dim))

In [6]:
colors.shape

(110592, 6)

In [7]:
decoded_colors.shape

(48, 48, 48)

In [8]:
encoded.shape

(48, 48, 48, 6)

In [124]:
from sklearn.preprocessing import LabelEncoder

## encoder ##

binarizer = LabelEncoder()

colors = binarizer.fit_transform(sculpture[1].reshape(-1, 1))

binary_encoded = np.array( [[int(char) for char in "{:03b}".format(color)] for color in colors] , dtype=object).reshape((48,48,48,3))

classes = binarizer.classes_

binary_encoded_array.shape

(48, 48, 48, 3)

In [125]:
output = binary_encoded_array

## decoder ##

dic_color = {
    '000':classes[0],
    '001':classes[1],
    '010':classes[2],
    '011':classes[3],
    '100':classes[4],
    '101':classes[5]
}

decoded = np.array([dic_color[key] for key in [''.join([str(num) for num in vector]) for vector in [i[0] for i in output.reshape(48*48*48,1,3).tolist()]]]).reshape((48,48,48))

In [127]:
decoded

array([[[None, None, None, ..., None, None, None],
        [None, None, None, ..., None, None, None],
        [None, None, None, ..., None, None, None],
        ...,
        [None, None, None, ..., None, None, None],
        [None, None, None, ..., None, None, None],
        [None, None, None, ..., None, None, None]],

       [[None, None, None, ..., None, None, None],
        ['dimgrey', 'dimgrey', 'dimgrey', ..., None, None, None],
        [None, None, None, ..., None, None, None],
        ...,
        [None, None, None, ..., None, None, None],
        ['dimgrey', 'dimgrey', 'dimgrey', ..., None, None, None],
        [None, None, None, ..., None, None, None]],

       [[None, None, None, ..., None, None, None],
        [None, None, None, ..., None, None, None],
        [None, None, None, ..., None, None, None],
        ...,
        [None, None, None, ..., None, None, None],
        [None, None, None, ..., None, None, None],
        [None, None, None, ..., None, None, None]],

       